<a href="https://colab.research.google.com/github/leohpark/Files/blob/main/Naive_vs_Contextual_Chunking_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compares Naive Chunking to Contextual Chunking for RAG Retrieval Comparisons

This app uses a Gradio interface that allows you to add your OpenAI and Pinecone credentials, as well as provide a PDF document. To work properly, the PDF must have text/ocr embedded already.

The Document is parsed two ways: using Langchain's RecursiveCharacterTextSplitter using user-configurable chunk sizes and the default delimiters, and via llmsherpa, with a bit of post-processing to establish a minimum and maximum total chunk size.

Each set of chunks is upserted to your Pinecone database using OpenAI's text-ada-002 embedding model via langchain. The first tab will display the overall chunking results once the upsert is complete.

NOTE: This notebook will OVERWRITE the designated Namespaces in your Pinecone Index. You can change the default Namespace values if you desire, but *DO NOT* provide a Namespace you would like to otherwise preserve. This is simulating an application where temporary vectors are created for document RAG querying, then discarded.

The default LLM in this notebook is GPT-4. If you have the base rate limit of 10,000 tokens/minute, then you may need to add some rate throttling, or downgrade to gpt-3.5-turbo-16k. Sorry for any inconvenience. Search for 'low-rate-limit' in this notebook to find the relevant function.

In [ ]:
!pip install -q gradio langchain unstructured pdf2image openai tiktoken pdfminer.six uuid pinecone-client llmsherpa

In [3]:
import gradio as gr
import openai
from llmsherpa.readers import LayoutPDFReader
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore import document
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

from google.colab import drive
import tiktoken
import pdfminer
import pinecone
import json, uuid, string, datetime, os, time, math

#RAG Retrieval and LLM Calls

Lanchain, mainly.

In [10]:
def rag_qa(user_settings, my_question, my_namespace):
  # for low-rate-limit gpt-4 users, you may need to add rate limiting somewhere in this function, or use gpt-3.5-turbo-16k.
  messages = []
  chat = ChatOpenAI(
      openai_api_key = user_settings['openai_key'],
      #model='gpt-3.5-turbo-16k',
      model = 'gpt-4',
      max_tokens=1024,
      temperature=0
  )
  my_vectors = get_vectors(user_settings, my_question, my_namespace)
  system_question = f"""Answer the Question based only on the facts and reasoning found in the Context. You are an analyst providing a detailed legal analysis to the question incorporating all of the information found in the Context.
If you cannot find information relevant to Question in the Context, say that the answer wasn't found.

Question: {my_question}"""

  context_prompt = SystemMessage(content=my_vectors)
  system_prompt = SystemMessage(content=system_question)
  messages.append(context_prompt)
  messages.append(system_prompt)
  answer = chat(messages)
  return answer.content, my_vectors

def get_vectors(user_settings, my_question, my_namespace):
  embeddings = OpenAIEmbeddings(openai_api_key=user_settings['openai_key'])
  pinecone.init(
      api_key = user_settings['pinecone_key'],
      environment=user_settings['pinecone_environment']
  )
  index = pinecone.Index(user_settings['pinecone_index'])
  vectorstore = Pinecone(index, embeddings, "text")
  my_retrieval = vectorstore.similarity_search(my_question, k=user_settings['top_k'], namespace=my_namespace)
  page_contents = [doc.page_content for doc in my_retrieval]
  rag_contents = ""
  for i, page in enumerate(page_contents, 1):
    rag_contents += f"Document {i}: " + page + "\n\n"
  return rag_contents

##Vector DB Bits

Langchain, Pinecone, OpenAI (text embeddings), LLM Sherpa

In [5]:
def tiktoken_len(text, base='cl100k_base'):
  tokenizer = tiktoken.get_encoding(base)
  tokens = tokenizer.encode(
      text,
      disallowed_special=()
  )
  return len(tokens)

def upsert_vectors(user_settings, chunks, namespace):
  embeddings = OpenAIEmbeddings(openai_api_key=user_settings['openai_key'])
  pinecone.init(
      api_key = user_settings['pinecone_key'],
      environment=user_settings['pinecone_environment']
  )
  index = pinecone.Index(user_settings['pinecone_index'])
  namespace_clear = check_namespace(user_settings, namespace)

  upsert = Pinecone.from_texts(chunks, embeddings, index_name=user_settings['pinecone_index'], namespace=namespace)

  chunks_list = ""
  for i, chunk in enumerate(chunks, 1):
    chunks_list += f"Chunk #{i}: " + "\n\n" + chunk + "\n\n"

  return chunks_list

#def contextual_vectors(user_settings, contextual_chunks)

def check_namespace(user_settings, test_namespace):
  pinecone.init(
      api_key=user_settings['pinecone_key'],
      environment=user_settings['pinecone_environment']
      )
  index = pinecone.Index(user_settings['pinecone_index'])
  index_stats = index.describe_index_stats()
  if test_namespace in index_stats['namespaces']:
    delete_response = index.delete(deleteAll='true', namespace=test_namespace)
    index_stats = index.describe_index_stats()
    if test_namespace in index_stats['namespaces']:
      raise Exception(f"Failed to delete namespace: {test_namespace}")
    return True
  return False


In [6]:
# @title Chunking Functions

def doc_upload(document):
    loader = UnstructuredPDFLoader(document.name)
    doc_text = loader.load()
    doc_content = doc_text[0].page_content[:]
    doc_tokens = tiktoken_len(doc_content)

    return document, doc_content, doc_tokens

def text_splitter(doc, max_tokens, overlap_tokens=0):
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size = int(max_tokens), #chunk_s, # number of units per chunk
      chunk_overlap = int(overlap_tokens), # number of units of overlap
      length_function = tiktoken_len, #use tokens as chunking unit instead of characters.
      separators=['\n\n', '\n'] # our chosen operators for separating
      )
  texts = text_splitter.split_text(doc)
  return texts

def combine_chunks(chunks, min_tokens):
    combined_chunks = []
    buffer_chunk = ""
    buffer_length = 0

    for chunk in chunks:
        chunk_text = chunk.to_context_text()  # Extract text representation of the chunk

        # Add newline if buffer_chunk already has content
        if buffer_chunk:
            buffer_chunk += "\n"

        buffer_chunk += chunk_text
        buffer_length += tiktoken_len(chunk_text)

        if buffer_length >= min_tokens:
            combined_chunks.append(buffer_chunk)
            buffer_chunk = ""
            buffer_length = 0

    # Add any remaining buffer_chunk to the list
    if buffer_chunk:
        combined_chunks.append(buffer_chunk)

    return combined_chunks

def split_and_prepend(chunks, max_tokens):
  #"""Split chunks exceeding the max token count and prepend the first line of the original chunk."""
  final_chunks = []
  for chunk in chunks:
    if tiktoken_len(chunk) > max_tokens:
      first_line = chunk.split("\n", 1)[0]
      split_chunks = text_splitter(chunk, max_tokens)
      # Prepend the first line to each split chunk
      split_chunks = [first_line + "\n" + sub_chunk for sub_chunk in split_chunks]
      final_chunks.extend(split_chunks)
    else:
      final_chunks.append(chunk)

  return final_chunks

def get_naive_chunks(user_settings):
  chunk_size = user_settings['chunk_size']
  #calculating 8% overlap, rounding up to nearest 10 tokens.
  chunk_raw = 0.08 * chunk_size
  chunk_overlap = math.ceil(chunk_raw / 10) * 10
  chunks = text_splitter(user_settings['doc_content'], chunk_size, chunk_overlap)

  return chunks

def get_contextual_chunks(user_settings):
  gradio_doc = user_settings['doc_doc']
  file_path = gradio_doc.name
  llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
  pdf_reader = LayoutPDFReader(llmsherpa_api_url)
  sherpa_doc = pdf_reader.read_pdf(file_path)
  combined_chunks = combine_chunks(sherpa_doc.chunks(), user_settings['context_chunk_min'])
  return split_and_prepend(combined_chunks, user_settings['context_chunk_max'])


##Front End to Back End Functions

In [7]:
def vectorize(
    openai_key, pinecone_key, pinecone_environment, pinecone_index,
    naive_namespace, contextual_namespace, doc_doc, doc_content, top_k, chunk_size,
    context_chunk_min, context_chunk_max):
  #make a dictionary to pass around
  user_settings = {
      'openai_key': openai_key,
      'pinecone_key': pinecone_key,
      'pinecone_environment': pinecone_environment,
      'pinecone_index': pinecone_index,
      'naive_namespace': naive_namespace,
      'contextual_namespace': contextual_namespace,
      'doc_doc': doc_doc,
      'doc_content': doc_content,
      'top_k': int(top_k),
      'chunk_size': int(chunk_size),
      'context_chunk_min': int(context_chunk_min),
      'context_chunk_max': int(context_chunk_max)
  }
  naive_chunks = get_naive_chunks(user_settings)
  contextual_chunks = get_contextual_chunks(user_settings)

  naive_v_text = upsert_vectors(user_settings, naive_chunks, user_settings['naive_namespace'])
  contextual_v_text = upsert_vectors(user_settings, contextual_chunks, user_settings['contextual_namespace'])

  return naive_v_text, contextual_v_text

def q_and_a(openai_key, pinecone_key, pinecone_environment, pinecone_index, naive_namespace, contextual_namespace,
            top_k, question_1=None, question_2=None, question_3=None):
  user_settings = {
    'openai_key': openai_key,
    'pinecone_key': pinecone_key,
    'pinecone_environment': pinecone_environment,
    'pinecone_index': pinecone_index,
    'naive_namespace': naive_namespace,
    'contextual_namespace': contextual_namespace,
    'top_k': int(top_k),
  }
      # Initialize default values
  naive_answer_1, naive_chunks_1, contextual_answer_1, contextual_chunks_1 = "", "", "", ""
  naive_answer_2, naive_chunks_2, contextual_answer_2, contextual_chunks_2 = "", "", "", ""
  naive_answer_3, naive_chunks_3, contextual_answer_3, contextual_chunks_3 = "", "", "", ""

  # Get values if questions are not None
  if question_1:
      naive_answer_1, naive_chunks_1 = rag_qa(user_settings, question_1, user_settings['naive_namespace'])
      contextual_answer_1, contextual_chunks_1 = rag_qa(user_settings, question_1, user_settings['contextual_namespace'])
  if question_2:
      naive_answer_2, naive_chunks_2 = rag_qa(user_settings, question_2, user_settings['naive_namespace'])
      contextual_answer_2, contextual_chunks_2 = rag_qa(user_settings, question_2, user_settings['contextual_namespace'])
  if question_3:
      naive_answer_3, naive_chunks_3 = rag_qa(user_settings, question_3, user_settings['naive_namespace'])
      contextual_answer_3, contextual_chunks_3 = rag_qa(user_settings, question_3, user_settings['contextual_namespace'])

  return (naive_answer_1, contextual_answer_1, naive_chunks_1, contextual_chunks_1,
          naive_answer_2, contextual_answer_2, naive_chunks_2, contextual_chunks_2,
          naive_answer_3, contextual_answer_3, naive_chunks_3, contextual_chunks_3)

  #results = {}
  #
  #questions = [question_1, question_2, question_3]
  #
  #for idx, question in enumerate(questions, 1):
  #  if question:
  #    naive_answer, naive_chunks = rag_qa(user_settings, question, naive_namespace)
  #    contextual_answer, contextual_chunks = rag_qa(user_settings, question, contextual_namespace)

  #    results[f'naive_answer_{idx}'] = naive_answer
  #    results[f'naive_chunks_{idx}'] = naive_chunks
  #    results[f'contextual_answer_{idx}'] = contextual_answer
  #    results[f'contextual_chunks_{idx}'] = contextual_chunks

  #return results

## Gradio App UI

In [8]:
with gr.Blocks() as demo:
  with gr.Row():
    with gr.Column(scale=3):
      with gr.Tab("Vector Chunking"):
        with gr.Accordion("API Keys and Pinecone Parameters"):
          gr.Markdown(
            """
            Set up your Keys and Configure Your Vector Store. DO NOT choose existing Pinecone Namespaces, as this App will delete and rewrite the Namespaces if they already exist.
            """)
          openai_key = gr.Textbox(scale=1, label="OpenAI API Key", placeholder='sk-...')
          pinecone_key = gr.Textbox(scale=1, label="Pinecone API Key", placeholder='...')

          with gr.Row():
            pinecone_environment = gr.Textbox(lines=1, label="Pinecone Environment", interactive=True, placeholder="us-west4-gcp-free")
            pinecone_index = gr.Textbox(lines=1, label="Index Name", interactive=True, placeholder="scotus")
          with gr.Row():
            naive_namespace = gr.Textbox(lines=1, label="Naive Chunking Namespace", interactive=True, value="my_pdf_naive_chunks")
            contextual_namespace = gr.Textbox(lines=1, label="Contextual Chunking Namespace", interactive=True, value="my_pdf_contextual_chunks")

        with gr.Row():
          input_doc_tokens = gr.Textbox(label="Tokens", scale=1)
          top_k = gr.Textbox(label="Top_K", value=3, scale=1)
          chunk_size = gr.Slider(100, 1500, value=600, step=50, label="Naive Chunk Size",
                                 info="Chunk Overlap will automatically be calculated to be approximately 8% of Chunk size", interactive=True, scale=4)
        with gr.Row():
          context_chunk_min = gr.Textbox(lines=1, value="400", max_lines=1, label="Context Chunk Minimum Size")
          context_chunk_max = gr.Textbox(lines=1, value="900", max_lines=1, label="Context Chunk Max Size")
        with gr.Row():
          naive_chunk_text = gr.Textbox(lines=15, max_lines=20, label="Naive Chunking", show_copy_button=True)
          contextual_chunk_text = gr.Textbox(lines=15, max_lines=20, label="Contextually Aware Chunks", show_copy_button=True)
        with gr.Row():
          upload_button = gr.UploadButton("Upload Doc", file_types=[".pdf"], file_count="single", size="sm")
          create_vectors_button = gr.Button("Create Vectors", variant="primary", size="sm")
          doc_doc = gr.State()
          doc_content = gr.State()

# Configure which options to include in Summaries
      with gr.Tab("Questions and Answers"):
        get_answers_button = gr.Button("Submit Questions", variant="primary", size="sm")
        question_1 = gr.Textbox(lines=2, max_lines=4, label="Question 1")
        with gr.Accordion("Question 1 Chunks and Answers", open=False):
          with gr.Row():
            naive_answer_1 = gr.Textbox(lines=15, max_lines=20, label="Naive Answer", show_copy_button=True)
            contextual_answer_1 = gr.Textbox(lines=15, max_lines=20, label="Contextual Answer", show_copy_button=True)
          with gr.Row():
            naive_chunks_1 = gr.Textbox(lines=15, max_lines=20, label="Naive Chunks", show_copy_button=True)
            contextual_chunks_1 = gr.Textbox(lines=15, max_lines=20, label="Contextual Chunks", show_copy_button=True)
        question_2 = gr.Textbox(lines=2, max_lines=4, label="Question 2")
        with gr.Accordion("Question 2 Chunks and Answers", open=False):
          with gr.Row():
            naive_answer_2 = gr.Textbox(lines=15, max_lines=20, label="Naive Answer", show_copy_button=True)
            contextual_answer_2 = gr.Textbox(lines=15, max_lines=20, label="Contextual Answer", show_copy_button=True)
          with gr.Row():
            naive_chunks_2 = gr.Textbox(lines=15, max_lines=20, label="Naive Chunks", show_copy_button=True)
            contextual_chunks_2 = gr.Textbox(lines=15, max_lines=20, label="Contextual Chunks", show_copy_button=True)
        question_3 = gr.Textbox(lines=2, max_lines=4, label="Question 3")
        with gr.Accordion("Question 3 Chunks and Answers", open=False):
          with gr.Row():
            naive_answer_3 = gr.Textbox(lines=15, max_lines=20, label="Naive Answer", show_copy_button=True)
            contextual_answer_3 = gr.Textbox(lines=15, max_lines=20, label="Contextual Answer", show_copy_button=True)
          with gr.Row():
            naive_chunks_3 = gr.Textbox(lines=15, max_lines=20, label="Naive Chunks", show_copy_button=True)
            contextual_chunks_3 = gr.Textbox(lines=15, max_lines=20, label="Contextual Chunks", show_copy_button=True)

  # Pinecone Setup Tab
  upload_button.upload(fn=doc_upload, inputs=[upload_button], outputs=[doc_doc, doc_content, input_doc_tokens])
  create_vectors_button.click(fn=vectorize, inputs=
   [openai_key, pinecone_key, pinecone_environment, pinecone_index,
    naive_namespace, contextual_namespace, doc_doc, doc_content, top_k, chunk_size,
    context_chunk_min, context_chunk_max], outputs=[naive_chunk_text, contextual_chunk_text]
                              )

  # Questions and Answers Tab
  get_answers_button.click(fn=q_and_a, inputs=[
      openai_key, pinecone_key, pinecone_environment, pinecone_index,
      naive_namespace, contextual_namespace, top_k, question_1, question_2, question_3],
                    outputs=[naive_answer_1, contextual_answer_1, naive_chunks_1, contextual_chunks_1,
                             naive_answer_2, contextual_answer_2, naive_chunks_2, contextual_chunks_2,
                             naive_answer_3, contextual_answer_3, naive_chunks_3, contextual_chunks_3,
                             ])

#Gradio Run

In [9]:
if __name__ == "__main__":
    demo.queue().launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://717db58f0f5a4fc983.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://717db58f0f5a4fc983.gradio.live
